In [7]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from ast import literal_eval
from cv_helper import CvHelper

## Load csv

In [130]:
output_folder_csv = "../output/csv"
video_name_0 = "010_001_test"
video_name_1 = "007_001_test"
video_name_2 = "007_001_test"
video_name_3 = "007_001_test"
video_name_4 = "007_001_test"


video_names = [video_name_0, video_name_1, video_name_2, video_name_3, video_name_4]

df_raw_angles_list = []

convert = {"trajectory_point_0_a": literal_eval,
           "trajectory_point_0_b": literal_eval,
           "trajectory_point_1_a": literal_eval,
           "trajectory_point_1_b": literal_eval,
           "trajectory_point_2_a": literal_eval,
           "trajectory_point_2_b": literal_eval,
           "trajectory_point_3_a": literal_eval,
           "trajectory_point_3_b": literal_eval}
for video_name in video_names:
    df_raw_angles_list.append(pd.read_csv(os.path.join(output_folder_csv, f"{video_name}.csv"), converters=convert))

## Load time intervlas json

In [131]:
time_coordinates_json_list = []
for video_name in video_names:
    with open(f"../output/time_coordinates/{video_name}.json", 'r') as file:
        time_coordinates_json_list.append(json.load(file))


## Prepair data lookups for one user each atemmpt moving

In [132]:
def get_user_finger_move_df(df_raw_angles_list, time_coordinates_json_list, user, direction):
    finger_move_attempt = []
    for measure_idx, interval in enumerate(time_coordinates_json_list[user]["loop_intervals"]):
        if direction == "flexion":
            df_time_lookup = df_raw_angles_list[user].loc[(df_raw_angles_list[user]["time"] > interval[0]) & (df_raw_angles_list[user]["time"] < interval[1])]
        elif direction  == "extension":
            df_time_lookup = df_raw_angles_list[user].loc[(df_raw_angles_list[user]["time"] > interval[1]) & (df_raw_angles_list[user]["time"] < interval[2])]
        else:
            print("NOT GOOD DIRECTION -> EXIT")

        # print(df_time_lookup.shape)
        for index, row in df_time_lookup.iterrows():
            rotated_points = CvHelper.rotate_frame_tracked_points(row["trajectory_point_0_a"],
                                                                  row["trajectory_point_0_b"],
                                                                  row["trajectory_point_1_b"],
                                                                  row["trajectory_point_2_b"],
                                                                      row["trajectory_point_3_b"])
            df_time_lookup.at[index,"trajectory_point_0_b"] = tuple(rotated_points[0])
            df_time_lookup.at[index,"trajectory_point_1_b"] = tuple(rotated_points[1])
            df_time_lookup.at[index,"trajectory_point_2_b"] = tuple(rotated_points[2])
            df_time_lookup.at[index,"trajectory_point_3_b"] = tuple(rotated_points[3])
            df_time_lookup.at[index,"pixel_distance"] = 5 / CvHelper.calculate_euclidean_distance(row["trajectory_point_0_a"],row["trajectory_point_0_b"])
        finger_move_attempt.append(df_time_lookup)
    return finger_move_attempt


## One user plot trajecotries
- Neccesary to have json file

In [198]:
which_user = 0

user_0_flexion_df_list = get_user_finger_move_df(df_raw_angles_list, time_coordinates_json_list, 0, "flexion")
user_0_extension_df_list = get_user_finger_move_df(df_raw_angles_list, time_coordinates_json_list, 0, "extension")

/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_20574/1418836923.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_20574/1418836923.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_20574/1418836923.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [199]:
from datetime import datetime
from scipy.interpolate import interp1d

def PlotOneUserMultipleAttempt(df_list, title_text, shot_legend=True):
    trajectory_point_names = ["trajectory_point_0_b", "trajectory_point_1_b", "trajectory_point_2_b", "trajectory_point_3_b"]# extension
    colors = ["blue", "pink", "green", "orange"]
    fig1 = go.Figure()
    for attempt_index, df_attempt in enumerate(df_list):
        for point_index, point_name in enumerate(trajectory_point_names):
            # Extract unique indices for data points
            pix_attempt_distance = np.mean(df_attempt["pixel_distance"])
            x_y_points_in_np_array = np.array([np.array(x) for x in df_attempt[point_name]])
            if point_index == 0:
                fig1.add_trace(go.Scatter(
                    x=x_y_points_in_np_array[:,0] * pix_attempt_distance  ,
                    y=(x_y_points_in_np_array[:,1]) * -1 * pix_attempt_distance,
                    mode='markers',
                    line=dict(color=colors[point_index]),
                    name=f'Attempt {attempt_index} - {point_name}'
                ))
            else:
                fig1.add_trace(go.Scatter(
                    x=x_y_points_in_np_array[:,0] * pix_attempt_distance,
                    y=(x_y_points_in_np_array[:,1]) * -1 * pix_attempt_distance,
                    mode='lines',
                    line=dict(color=colors[point_index]),
                    name=f'Attempt {attempt_index} - {point_name}'
                ))
    fig1.update_layout(
        title={
            'text': f"User {which_user}, {title_text}",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title='X-axis (cm)',
        yaxis_title='Y-axis (cm)',
        xaxis=dict(range=[-1, 15],dtick=1),  # Set the x-axis range
        yaxis=dict(range=[-10, 5],dtick=1),   # Set the y-axis range
        height=500,
        width=800,
        showlegend=shot_legend

    )
    # Get the current date and time
    current_datetime = datetime.now()

    # Format the date and time as a string
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

    fig1.write_image(f"../output/plots/PlotOneUserMultipleAttempt_user_{which_user}_{title_text}_{formatted_datetime}.png")
    fig1.show()


In [200]:
PlotOneUserMultipleAttempt(user_0_flexion_df_list, "test_flexion", shot_legend=True)

In [202]:
PlotOneUserMultipleAttempt(user_0_extension_df_list, "test_extension", shot_legend=False)